In [176]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from matplotlib import pyplot as plt

In [197]:
class Generator(nn.Module):
    # 생성모델 정의
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256, normalize=False),
            *block(256, 512, normalize=False),
            *block(512, 258, normalize=False),
            nn.Linear(258, 166),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.model(x)
        return x


class Discriminator(nn.Module):
    # 식별모델 정의
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(166, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        validity = self.model(x)

        return validity


def extract(v):
    return v.data.storage().tolist()

def stats(d):
    return [np.mean(d), np.std(d)]


df = pd.read_csv("../train.csv")
# hyper parameter
G = Generator()
D = Discriminator()
loss = nn.BCELoss()
epochs = 10000
d_optimizer = optim.Adam(D.parameters(), lr=1e-3, betas=(0.1, 0.9))
g_optimizer = optim.Adam(G.parameters(), lr=1e-3, betas=(0.1, 0.9))
col_counts = len(df.columns)
d_steps = 20
g_steps = 20

for epoch in range(epochs):
    for d in range(d_steps):
        d_real_data = torch.tensor(df.iloc[1, ].values).float()
        d_real_decision = D(d_real_data)
        d_real_error = loss(d_real_decision, torch.ones([1]))
        d_real_error.backward()
        
        d_gen_input = Variable(torch.rand(64))
        d_fake_data = G(d_gen_input)
        d_fake_decision = D(d_fake_data.t())
        d_fake_error = loss(d_fake_decision, Variable(torch.zeros([1])))
        d_fake_error.backward()
        d_optimizer.step()

        dre, dfe = extract(d_real_error)[0], extract(d_fake_error)[0]

    for g in range(g_steps):
        G.zero_grad()
        
        gen_input = Variable(torch.rand(64))
        g_fake_data = G(gen_input)
        dg_fake_decision = D(g_fake_data.t())
        g_error = loss(dg_fake_decision, torch.ones([1]))
        
        g_error.backward()
        g_optimizer.step()
        ge = extract(g_error)[0]
        
        if epoch % 100 == 0:
            print("Epoch %s: D (%s real_err, %s fake_err) G (%s err); Real Dist (%s),  Fake Dist (%s) " %
            (epoch, dre, dfe, ge, stats(extract(d_real_data)), stats(extract(d_fake_data))))
            print(f'True_data[0]: {d_real_data}, \n Fake_data[0]: {g_fake_data}')

Epoch 0: D (0.0 real_err, 0.00018944150360766798 fake_err) G (9.261293411254883 err); Real Dist ([28844.096967396337, 82694.33422128939]),  Fake Dist ([0.00280569971774716, 0.026400366859895438]) 
True_data[0]: tensor([1.0000e+00, 1.1500e+01, 2.7543e+01, 7.8075e+01, 5.2399e+02, 1.3844e+00,
        3.9806e+03, 2.5632e+05, 1.3708e+04, 0.0000e+00, 2.7003e+05, 1.8179e-01,
        1.0000e+00, 1.9583e+00, 3.9967e+00, 9.0000e+00, 1.9000e+01, 7.9418e+03,
        5.1200e+05, 0.0000e+00, 2.7407e+04, 5.3941e+05, 5.0925e+02, 5.1575e+02,
        5.2285e+02, 5.2944e+02, 5.3630e+02, 5.4392e+02, 5.5092e+02, 5.5631e+02,
        5.5910e+02, 5.5869e+02, 5.5506e+02, 5.4831e+02, 5.3966e+02, 5.2970e+02,
        5.1975e+02, 5.1424e+02, 5.0806e+02, 5.0271e+02, 4.9876e+02, 4.9651e+02,
        4.9623e+02, 4.9496e+02, 4.9728e+02, 5.0100e+02, 1.3838e+00, 1.3861e+00,
        1.3886e+00, 1.3906e+00, 1.3923e+00, 1.3937e+00, 1.3950e+00, 1.3949e+00,
        1.3942e+00, 1.3931e+00, 1.3912e+00, 1.3888e+00, 1.3860e+00, 1

KeyboardInterrupt: 

In [189]:
d_real_data = torch.tensor(df.iloc[1, ].values).float()
d_real_data[0]

tensor(1.)